<a href="https://colab.research.google.com/github/thad75/OptionAI/blob/main/TP4/TPSR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spotipy
!pip install pytorch_lightning
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import pandas as pd
import torch
from torch.utils.data import Dataset
import scipy.sparse as sp
import pandas as pd
import torch.nn.functional as F
import torch
torch.manual_seed(0)
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

# TP4 : Spotify Recommendation System

As a future Deep Learning engineer, you will be asked to tackle some fundamental tasks. Recommendation Systems is one of these fundamental task. Obviously, you had a course introducing you to the world of recommendation. Hence, we will create a recommendation system based on Spotify Data. 
<div align='center'>
<img src="https://thumbs.dreamstime.com/b/spotify-logo-mode-fonc%C3%A9-vinnytsia-ukraine-mai-service-de-musique-cool-qui-offre-la-en-streaming-l%C3%A9gale-218033923.jpg" width=400></div>

**The goal is to create a Playlist recommendation system.**


## Outline

Goal of this lab:
* Use an API
* **Read Documentation** 
* Create a functioning model
* Use Pytorch-Lightning
* See all the roles a Data Scientist can take

This lab is written to introduce you to a real-case application of RecSys



## Important : Set your Credentials 

Before beginning the lab make sure that you set your Spotify API Credentials correctly. If you don't have any credentials, please refer to TDm 1 and retrieve your keys.

In [ ]:
# TODO : Set your credentials
id = ...
secret = ...
# Set up Spotipy client credentials
client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Introduction


You are working as a Data Scientist in FlexCorp, a startup that makes a Recommendation System. Obviously, as any flexible companies, an Intern was hired and did some stuff. And obviouly, there are loads of errors in it. Let's correct and give a better product.

You will be roaming around the different roles a Data Scientist can have in company. Your mission revolves around two things:
* Correct the existing works
* Fullfill the specification of the company

The company uses Pytorch-Lightning as their DL Framework

# I - Extract, Transform & Load : Being a Data Engineer and Data Analyst

Well, let's explore some data from Spotify. Obviously, we need some data. The intern did do some work on it. Let's check the usefulness and correct the errors.

* Code Review: Correct the intern's work


### A - Data Engineering : Retrieving and Cleaning Data 



---
Company's specification:
* Retrieve Data from 1960 to 2023 in a list.

---



In [ ]:
# TODO : Correct the intern's code 

query_res = [sp.search(q='year:' + str(i), type='track', limit=50) for i in range(2022, 2023)]
tracks = []
for result in query_res:
    tracks += result['tracks']['items']
print(tracks)


---
Company's specification:
* Retrieve the following keys is a dictionnary:
  * user_id
  * song_id
  * danceability
  * energy
  * speechiness
  * loudness
  * tempo

* Create a pandas dataframe

---

In [ ]:
track = sp.audio_features(tracks[0]['id'])[0]
track['tempo']

In [ ]:
# TODO : Extract relevant features from tracks
data = []
for i in range(len(tracks)):
    track = sp.audio_features(tracks[i]['id'])[0]
    # Hint : verify the existence of the keys in track[i]
    data.append({
        'user_id': 'user_{}'.format(i % 10),
        'song_id': tracks[i]['id'],
        'danceability': track['danceness'],
        'energy': track['red bull'],
        'speechiness': track['je mapelle JP'],
        'loudness': track['loudness'],
        'tempo': int(track['1+1 = 42'])
    })

# TODO : Create a dataset
df = ...

* What do you think of the Intern's work ? Is he hireable ?

## B - Data Analyst : Plotting Some Stuffs to Understand the Data





### Data : Speak to Me !

So we have a dataframe, but before training any models, we need to understand the data. 

Let's perform Exploratory Data Analysis
* Perform the Data Exploration

In [ ]:
# TODO : Print your dataset


In [ ]:
# TODO : What informations does this code gives you ?
df.describe()

In [ ]:
# TODO : Print the unique songs and users.
num_users = ...
num_songs = ...
print('Number of unique users:', num_users)
print('Number of unique songs:', num_songs)

In [ ]:
# TODO : Is there any outliers in the DataFrame ? Check for nan values
print(...)

In [ ]:
# TODO : Plot the distribution of each features. 
df.hist(figsize=(10,10))
plt.show()

In [ ]:
# TODO : Using Seaborn plot the pairwise distributions of each features.
sns.pairplot(df[...])
plt.show()


In [ ]:
# TODO : Plot the top 10 songs features sorted by their 'danceability'. Hint : GroupBy is your friend.
top_songs = ...
print(top_songs)


In [ ]:
# TODO : Plot the top 10 users feautures sorted by their danceab
top_users = ...
print(top_users)


Normally, we are supposed to have a lot more understanding of the Dataset now. But there are few issues that might come from the data as is.
* What further steps do we need on the dataset ?

### Creating a Rating

As seen before, there's no specific ratings here. We can create one based on the other features. But we need to preprocess a little bit the existing features.
* Do we need to perform preprocessing on each features ?
* What type of preprocessing do we need to perform on the selected features ?
* What should it change on the data ?
* Using MinMaxScale, do the needed preprocessing.

More documentation on :  https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# TODO : Define the columns to normalize. Hint : It must be a list of strings
cols_to_normalize = [...]
# TODO : Rescale and Replace the values apres preprocessing.
df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])


Now we can compute a rating based on other features. For the moment, we will just take in account 2 features for the computation of the rating value:   **Energy, Loudness**

* When computing the rating value, do we need to re-do some post-processing on it ?

  

In [ ]:
# TODO : Create your ratings
df['rating'] = ...


In [ ]:
# TODO : Using .describe(), describe your new dataframe. What can you tell about the rating column ?
...

In [ ]:
# TODO : Calculate the mean rating for each user. Don't forget to groupby
user_ratings = ...

# TODO : Calculate the mean rating for each song
song_ratings = ...


### Data : Speak to the Computer !

We're nearly over with the Data Engineering stuff. But we need to change something crucial about the existing user/items values.

* What is the issue with the current form of the Song_id and User_id values ?
* As seen in class, what preprocessing do we need to do on these specific columns ?
* Do we need to create separate pre-processors to re-encode the user and song columns ? Why ?

In [ ]:
# TODO : Print the user_id or song_id. What is the issue ?

In [ ]:
from sklearn.preprocessing import LabelEncoder

# TODO : Re-encode the user_id and song_id columns to the correct values.
encoder_user = ...
encoder_song = ...

df.user_id = ...
df.song_id = ...

## C - Creating the PyTorch Dataset: Being a ML/DL Ops



#### Creating the PyTorch Dataset

That was a long ride. But we are ready to train our model now. But first, let's create a Pytorch Dataset Class that we can reuse.
The PlaylistDataset class takes as input:
* df : the dataframe you've been creating from the beginning
Now let's get into the details of this class : 


* Fill in the blanks

In [ ]:
class PlaylistDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # TODO : Retrieve the row corresponding to idx from the dataframe
        row = ...
        # TODO : Retrieve these values and convert them into torch.tensor. Hint : Really be careful on the type of everything..
        user = torch.tensor(row['user_id'],dtype=torch.int) # Example
        song = ...
        danceability = ...
        energy = ...
        speechiness = ...
        loudness = ...
        tempo = ...
        rating = ...


        return {
            'user': user,
            'song': song,
            'danceability': danceability,
            'energy': energy,
            'speechiness': speechiness,
            'loudness': loudness,
            'tempo': tempo,
            'rating': rating
        }


In [ ]:
# TODO : Make sure that your Dataset returns what we want.
dataset = ...
...

#### Creating the PyTorch Lightning Datamodule

Obviously, we can still rewrite all the code we wrote each and every time to train a model but meh, it's too much work :)

Let's use Pytorch Lightning.

* The DataModule class is a simple wrapper that encompasses all your datasets and dataloaders under one class. However, we need some proper initialization.
We need to create the Training, Validation and Testing Dataloader for each data sets we have.

* Fill in the blanks

In [ ]:
class PlaylistDataModule(pl.LightningDataModule):
    def __init__(self, df, batch_size=32, val_ratio=0.3):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.val_ratio = val_ratio
        
    def prepare_data(self):
        # download or preprocess data here
        pass
        
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            # TODO : Split dataset into train and val sets
            train_len = ...
            val_len = ...
            self.train_data, self.val_data = random_split(..., [train_len, val_len]) # TODO : Google Random_split, and find what you have to put here
        if stage == 'test' or stage is None:
            # use entire dataset for testing
            self.test_data = self.df
            
    def train_dataloader(self):
        # TODO : Init your train_loader
        train_loader = DataLoader(..., batch_size=..., shuffle=True)
        return train_loader
    
    def val_dataloader(self):
        # TODO : Init your val_loader
        val_loader = DataLoader(...,..., shuffle=False)
        return val_loader
    
    def test_dataloader(self):
        # TODO : Init your test_loader
        test_loader = DataLoader(...,..., shuffle=False)
        return test_loader


# II - Getting Back to the NCF with Lightning : Being a Data Scientist

As seen in class, we have plenty of models available to train a RecSys model. Let's use a model everyone uses: Neural Collaborative Filtering Model.

However, comparing it to the one presented in class, the present model is a little bit more complex version that will handle more features than the basic User/Items embedding layers of the NCF.
* What are the two mains parts of the NCF model ?
* What is the purpose of the nn.Embedding layer ?
* What are the differences between the Embedding Layers and the Linear Layers.
* What basic operation do we perform to keep the informations given by all the features with User Embedding and Item Embedding ?


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

class NCF(pl.LightningModule):
    def __init__(self, num_users, num_songs, embed_dim=32, hidden_dim=64, used_features=2):
        super(NCF, self).__init__()
        
        # User embedding layer
        self.user_embed = nn.Embedding(num_users, embed_dim)        
        # TODO : Define your Song embedding layer
        self.song_embed = ...
        # TODO : Define your MLP layers. The first layer is given
        self.layer1 = nn.Linear(embed_dim*2+used_features, hidden_dim)
        self.layer2 = ...
        self.layer3 = ...
        
        # TODO : What's the purpose of this class attribute ?
        self.used_features = used_features 

    def forward(self, user, song,  energy, loudness,danceability= None, speechiness= None ,  tempo= None):
        # TODO : Embed user and song
        user_embed = self.user_embed(...)
        song_embed = self.song_embed(...)
        
        # TODO : Perform the basic operation. Hint : Don't forget to unsqueeze some things if shapes doesn't fit..
        x = ...
        
        # TODO : Pass through MLP layers
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        
        # Output final prediction
        return x.view(-1)
        
    def training_step(self, batch, batch_idx):
        # TODO : Retrieve User, Song, Energy, Loudness, Rating from the input batch
        user = ...
        song = ...
        energy = ...
        loudness = ...
        rating = ...        
        # TODO : Forward pass
        output = self.forward(...,...,...,...)        
        # TODO : Compute your loss. Hint : Do we use a MSE, BCE, CE, Focal Loss ?.
        loss = ...
        # Log training loss
        self.log('train_loss', loss)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        # TODO : What changes do we have to make from the training step ? Can we just copy paste stuff ?
        # TODO : Add what's needed.. You've done this at least 3 times now
        ...


    def configure_optimizers(self):
        # TODO : Define your learning_rate
        optimizer = torch.optim.Adam(self.parameters(), lr= ...)
        return optimizer



### Instantiate the model

Now let's create an instance of the model and the datamodule.

In [ ]:

# TODO : Initialize model
model = NCF(...,...)

# TODO : Initialize datamodule
datamodule = PlaylistDataModule(...,...)

### Trainer : The magic of Lightning

So we have the choice to either recreate the famous training, valid and testing loop, but meh..

Lightning comes with a Trainer class that handles all the training for your.
* By reading the documentation: https://lightning.ai/docs/pytorch/stable/common/trainer.html, initiate the attributes of the class.
* Train and test the model on the dataset.

In [ ]:
# TODO : Initialize PyTorch Lightning Trainer
trainer = pl.Trainer(accelerator=...,
                     devices=...,
                     max_epochs=...,
                     fast_dev_run=...)

# TODO : Train the model
datamodule.setup(...) # TODO : Setup your datamodule to the training mode
.. # TODO : TRAIN !!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Evaluate the model on the test set
... # TODO : TEST !!!!!!!!!!!!!!!!!!!!!!!!!!!!


### Monitoring the Training

Obviously, we have tools that allows us to monitor the training of our model. 
* Initiate a Tensorboard and watch your training.

If you have issues showing the Tensorboard UX, you're cookies might not allow it. So try to activate them.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ...

### Test the predictions

* Create a Playlist for user 1 based on the ratings your model predicted. You can obviously retrieve the top10 songs. But how ? :)

* The playlist should return the name of the song and its rating

In [ ]:
def predict(model, user_id, song_ids):
    ratings = ...            
    return ratings.numpy()

# TODO : Retrieve the TOP10 songs for user 1.

# III - Doing Some Feature Engineering : The Complete Package (Optional)

As you might have seen, we only kept two features (Energy, Loudness) to compute the final Rating . But, we can use much more than that. As we have remaining features (Danceability..), we can add them to our rating computation, and explain their contribution on the prediction of our model.
* Explain the impact of the other features on the training of the model.
* Train (multiple?) models to validate your hypothesis.
